Credit to Lori Garzio for the soucre code in calculating the mixed layer depth on her GitHub - https://github.com/lgarzio/gliders/blob/master/analyses/calculate_mld.py

In [1]:
#!/usr/bin/env python

"""
Author: Lori Garzio on 1/11/2022
Last modified: 11/8/2023
Modified from code from Sam Coakley following theory from Carvalho et al 2016 https://doi.org/10.1002/2016GL071205
Calculate Mixed Layer Depth for glider profiles using density and pressure, then add the MLD variable to the .nc file.
The dataset provided must have 'time' or 'profile_time' as the only coordinate in order to convert the dataset to a
dataframe properly
"""

import os
import datetime as dt
import xarray as xr
import numpy as np
import pandas as pd
import gsw
import matplotlib.pyplot as plt
# import functions.common as cf
# import functions.mixed_layer_depth as mldfunc
plt.rcParams.update({'font.size': 14})
pd.set_option('display.width', 320, "display.max_columns", 20)  # for display in pycharm console

In [2]:
#! /usr/bin/env python3

"""
Author: Lori Garzio on 10/23/2023
Last modified: 10/26/2023
"""
import numpy as np
import pandas as pd
import cmocean as cmo


def depth_bin(dataframe, depth_var='depth', depth_min=0, depth_max=None, stride=1):
    """
    Written by Mike Smith
    :param dataframe: depth profile in the form of a pandas dataframe
    :param depth_var: the name of the depth variable in the dataframe
    :param depth_min: the shallowest bin depth
    :param depth_max: the deepest bin depth
    :param stride: the amount of space between each bin
    :return: pandas dataframe where data has been averaged into specified depth bins
    """
    depth_max = depth_max or dataframe[depth_var].max()
    print(dataframe[depth_var])

    bins = np.arange(depth_min, depth_max+stride, stride)  # Generate array of depths you want to bin at
    cut = pd.cut(dataframe[depth_var], bins)  # Cut/Bin the dataframe based on the bins variable we just generated
    binned_df = dataframe.groupby(cut, observed=False).mean()  # Groupby the cut and do the mean
    return binned_df


def glider_extent(lats, lons):
    """
    Calculate the map extents for plotting a glider deployment
    """
    extent = [np.nanmin(lons) - 2, np.nanmax(lons) + 2,
              np.nanmin(lats) - 1.5, np.nanmax(lats) + 1.5]

    return extent

In [3]:
#! /usr/bin/env python3

import numpy as np


def gap(prange):
    """
    :param prange: pressure range of the profile
    :return: the maximum allowable data gap for the profile
    """
    if prange < 20:
        gap_threshold = 8
    elif 20 <= prange < 50:
        gap_threshold = 10
    elif 50 <= prange < 200:
        gap_threshold = 25
    elif 200 <= prange < 500:
        gap_threshold = 50
    else:
        gap_threshold = 75

    return gap_threshold


def profile_mld(df, mld_var='density', zvar='pressure', qi_threshold=0.5):
    """
    Written by Sam Coakley and Lori Garzio, Jan 2022
    Calculates the Mixed Layer Depth (MLD) for a single profile as the depth of max Brunt‐Vaisala frequency squared
    (N**2) from Carvalho et al 2016 (https://doi.org/10.1002/2016GL071205). Calculates a Quality Index to determine
    if the water column is well-mixed, hence calculating MLD is not appropriate and the function will return
    MLD = np.nan, from Lorbacher et al, 2006 doi:10.1029/2003JC002157. When "QI > 0.8 a well defined MLD results.
    For QI in the range 0.5– 0.8, increased uncertainty of the profile interpretation becomes evident and with
    QI < 0.5 no mixed layer interpretation is possible." (from Section 3.4)
    :param df: depth profile in the form of a pandas dataframe
    :param mld_var: the name of the variable for which MLD is calculated, default is 'density'
    :param zvar: the name of the depth variable in the dataframe, default is 'pressure'
    :param qi_threshold: quality index threshold for determining well-mixed water, default is 0.5
    :return: the depth of the mixed layer in the units of zvar and the max buoyancy frequency in units of s-2
    """
    df.dropna(subset=[mld_var], inplace=True)
    pN2 = np.sqrt(9.81 / np.nanmean(df[mld_var]) * np.diff(df[mld_var], prepend=np.nan) / np.diff(df[zvar], prepend=np.nan)) ** 2
    if len(df) < 5:  # if there are <5 data bins, don't calculate MLD
        mld = np.nan
        maxN2 = np.nan
        qi = np.nan
    elif np.sum(~np.isnan(pN2)) < 3:  # if there are <3 values calculated for pN2, don't calculate MLD
        mld = np.nan
        maxN2 = np.nan
        qi = np.nan
    elif np.nanmax(np.diff(df[zvar])) > gap(np.nanmax(df[zvar]) - np.nanmin(df[zvar])):
        # if there is a gap in the profile that exceeds the defined threshold, don't calculate MLD
        mld = np.nan
        maxN2 = np.nan
        qi = np.nan
    else:
        pressure_range = [np.nanmin(df[zvar]), np.nanmax(df[zvar])]

        maxN2 = np.nanmax(pN2)
        mld_idx = np.where(pN2 == maxN2)[0][0]

        # if the code finds the first or last data point to be the max pN2, return nan
        if np.logical_or(mld_idx == 0, mld_idx == len(df) - 1):
            mld = np.nan
            maxN2 = np.nan
            qi = np.nan
        else:
            mld = np.nanmean([df[zvar][mld_idx], df[zvar][mld_idx + 1]])

            if mld < 5:
                # if MLD is <5, return nan
                mld = np.nan
                maxN2 = np.nan
                qi = np.nan
            elif np.logical_or(mld < pressure_range[0] + 2, mld > pressure_range[1] - 2):
                # if MLD is within 2 dbar of the top or bottom of the profile, return nan
                mld = np.nan
                maxN2 = np.nan
                qi = np.nan
            else:
                if qi_threshold:
                    # find MLD  1.5
                    mld15 = mld * 1.5
                    mld15_idx = np.argmin(np.abs(df[zvar] - mld15))

                    # Calculate Quality index (QI) from Lorbacher et al, 2006 doi:10.1029/2003JC002157
                    surface_mld_values = df[mld_var][0:mld_idx]  # values from the surface to MLD
                    surface_mld15_values = df[mld_var][0:mld15_idx]  # values from the surface to MLD * 1.5

                    qi = 1 - (np.std(surface_mld_values - np.nanmean(surface_mld_values)) /
                              np.std(surface_mld15_values - np.nanmean(surface_mld15_values)))

                    if qi < qi_threshold:
                        # if the Quality Index is < the threshold, this indicates well-mixed water so don't return MLD
                        mld = np.nan
                        maxN2 = np.nan

    return mld, maxN2, qi

In [4]:
def main(fname, timevar, plots, mldvar, zvar):
    mld = np.array([], dtype='float32')
    max_n2 = np.array([], dtype='float32')

    savefile = f'{fname.split(".nc")[0]}_mld.nc'

    ds = xr.open_dataset(fname)
    ds = ds.sortby(ds.time)
    deploy = ds.title

    df = ds.to_dataframe()

    # remove data (pressure plus the variable you're using to calculate MLD) that's collected at the surface (< 1 dbar)
    df.loc[df.pressure < 1, ['pressure', mldvar]] = np.nan
    grouped = df.groupby(timevar, dropna=False)

    if plots:
        plots = os.path.join(plots, 'mld_analysis', deploy)
        os.makedirs(plots, exist_ok=True)

    for group in grouped:
        # Create temporary dataframe to interpolate to dz m depths
        ll = len(group[1])
        kwargs = {'depth_var': zvar}
        temp_df1 = group[1][[mldvar, zvar, 'temperature']].dropna(how='all')
        if len(temp_df1) == 0:
            mldx = np.repeat(np.nan, ll)
            max_n2x = np.repeat(np.nan, ll)
        else:
            temp_df = depth_bin(temp_df1, **kwargs)
            temp_df.dropna(subset=[mldvar], inplace=True)
            temp_df.index.name = f'{zvar}_bins'
            temp_df.reset_index(inplace=True)
            if len(temp_df) == 0:
                mldx = np.repeat(np.nan, ll)
                max_n2x = np.repeat(np.nan, ll)
                qi = 'MLD not calculated'
            else:
                # calculate profile's pressure range
                pressure_range = (np.nanmax(temp_df[zvar]) - np.nanmin(temp_df[zvar]))

                if pressure_range < 5:
                    # if the profile spans <5 dbar, don't calculate MLD
                    mldx = np.repeat(np.nan, ll)
                    max_n2x = np.repeat(np.nan, ll)
                    qi = 'MLD not calculated'

                else:
                    kwargs = {'zvar': zvar}
                    mldx, N2, qi = profile_mld(temp_df, **kwargs)
                    mldx = np.repeat(mldx, ll)
                    max_n2x = np.repeat(N2, ll)

            if plots:
                try:
                    tstr = group[0].strftime("%Y-%m-%dT%H%M%SZ")
                except AttributeError:
                    tstr = pd.to_datetime(np.nanmin(group[1].time)).strftime("%Y-%m-%dT%H%M%SZ")
                # plot temperature
                fig, ax = plt.subplots(figsize=(8, 10))
                ax.scatter(temp_df['temperature'], temp_df[zvar])

                ax.invert_yaxis()
                ax.set_ylabel('Pressure (dbar)')
                ax.set_xlabel('temperature')

                ax.axhline(y=np.unique(mldx), ls='--', c='k')

                sfile = os.path.join(plots, f'temperature_{tstr}.png')
                plt.savefig(sfile, dpi=300)
                plt.close()

                # plot density
                fig, ax = plt.subplots(figsize=(8, 10))
                ax.scatter(temp_df['density'], temp_df[zvar])

                ax.invert_yaxis()
                ax.set_ylabel('Pressure (dbar)')
                ax.set_xlabel('density')
                ax.set_title(f'QI = {qi}\nN2 = {np.unique(max_n2x)[0]}')

                ax.axhline(y=np.unique(mldx), ls='--', c='k')

                sfile = os.path.join(plots, f'density{tstr}.png')
                plt.savefig(sfile, dpi=300)
                plt.close()

        mld = np.append(mld, mldx)
        max_n2 = np.append(max_n2, max_n2x)

    # add mld to the dataset
    mld_min = np.nanmin(mld)
    mld_max = np.nanmax(mld)
    attrs = {
        'actual_range': np.array([mld_min, mld_max]),
        'ancillary_variables': [mldvar, zvar],
        'observation_type': 'calculated',
        'units': ds[zvar].units,
        'comment': 'Mixed Layer Depth calculated as the depth of max Brunt‐Vaisala frequency squared (N**2) '
                   'from Carvalho et al 2016 (https://doi.org/10.1002/2016GL071205)',
        'long_name': 'Mixed Layer Depth'
        }
    da = xr.DataArray(mld, coords=ds[mldvar].coords, dims=ds[mldvar].dims,
                      name='mld_dbar', attrs=attrs)
    ds['mld_dbar'] = da

    # calculate MLD in meters
    mld_meters = gsw.z_from_p(-ds.mld_dbar.values, ds.latitude.values)

    attrs = {
        'actual_range': np.array([np.nanmin(mld_meters), np.nanmax(mld_meters)]),
        'observation_type': 'calculated',
        'units': 'm',
        'comment': 'Mixed Layer Depth calculated as the depth of max Brunt‐Vaisala frequency squared (N**2) from '
                   'Carvalho et al 2016 (https://doi.org/10.1002/2016GL071205). Calculated from MLD in dbar and latitude using gsw.z_from_p',
        'long_name': 'Mixed Layer Depth'
    }
    da = xr.DataArray(mld_meters, coords=ds.mld_dbar.coords, dims=ds.mld_dbar.dims,
                      name='mld', attrs=attrs)
    ds['mld'] = da

    # add maximum buoyancy frequency N2 (measure of stratification strength) to the dataset
    n2_min = np.nanmin(max_n2)
    n2_max = np.nanmax(max_n2)
    attrs = {
        'actual_range': np.array([n2_min, n2_max]),
        'ancillary_variables': [mldvar, zvar],
        'observation_type': 'calculated',
        'units': 's-2',
        'comment': 'Maximum Brunt‐Vaisala frequency squared (N**2) for each profile used to calculate Mixed Layer Depth '
                   'from Carvalho et al 2016 (https://doi.org/10.1002/2016GL071205). This can be used as a measurement '
                   'for stratification strength',
        'long_name': 'Maximum Buoyancy Frequency'
    }
    da = xr.DataArray(max_n2, coords=ds[mldvar].coords, dims=ds[mldvar].dims,
                      name='max_n2', attrs=attrs)
    ds['max_n2'] = da

    ds.to_netcdf(savefile)

In [5]:
if __name__ == '__main__':
    ncfile = '/Users/nixo1/Desktop/MLD_results/ru39-20240215T1646-profile-sci-delayed_slice2.nc'  # striper-20170907T1430.nc ru30-20180705T1825.nc
    time_variable = 'profile_time'  # time variable on which groups are generated (e.g. profile_time)
    generate_plots = False # '/Users/garzio/Documents/rucool/Saba/gliderdata/2023/ru39-20230817T1520/delayed/ncei'  # False or save_directory e.g. '/Users/garzio/Documents/rucool/Saba/gliderdata/plots'
    mldvar = 'density'  # variable used to calculate MLD
    zvar = 'pressure'  # pressure variable
    main(ncfile, time_variable, generate_plots, mldvar, zvar)

row
0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
197    54.360001
198    54.770000
199    55.189999
200    55.439999
201    55.470001
Name: pressure, Length: 202, dtype: float32
row
202    55.470001
203    55.430000
204    55.459999
205    55.470001
206    55.380001
         ...    
387     2.830000
388     2.570000
389     2.430000
390     2.260000
391     2.220000
Name: pressure, Length: 190, dtype: float32
row
392     2.220000
393     2.260000
394     2.320000
395     2.460000
396     2.730000
         ...    
582    54.730000
583    54.860001
584    55.070000
585    55.259998
586    55.290001
Name: pressure, Length: 195, dtype: float32
row
587    55.290001
588    55.209999
589    55.070000
590    54.840000
591    54.590000
         ...    
778     2.710000
779     2.510000
780     2.260000
781     2.110000
782     2.040000
Name: pressure, Length: 196, dtype: float32
row
783     2.040000
784     2.060000
785     2.140000
786  

C:\Users\nixo1\AppData\Local\Temp\ipykernel_22600\3273644669.py:41: RuntimeWarning: invalid value encountered in sqrt
  pN2 = np.sqrt(9.81 / np.nanmean(df[mld_var]) * np.diff(df[mld_var], prepend=np.nan) / np.diff(df[zvar], prepend=np.nan)) ** 2
C:\Users\nixo1\AppData\Local\Temp\ipykernel_22600\3273644669.py:41: RuntimeWarning: invalid value encountered in sqrt
  pN2 = np.sqrt(9.81 / np.nanmean(df[mld_var]) * np.diff(df[mld_var], prepend=np.nan) / np.diff(df[zvar], prepend=np.nan)) ** 2
C:\Users\nixo1\AppData\Local\Temp\ipykernel_22600\3273644669.py:41: RuntimeWarning: invalid value encountered in sqrt
  pN2 = np.sqrt(9.81 / np.nanmean(df[mld_var]) * np.diff(df[mld_var], prepend=np.nan) / np.diff(df[zvar], prepend=np.nan)) ** 2
C:\Users\nixo1\AppData\Local\Temp\ipykernel_22600\3273644669.py:41: RuntimeWarning: invalid value encountered in sqrt
  pN2 = np.sqrt(9.81 / np.nanmean(df[mld_var]) * np.diff(df[mld_var], prepend=np.nan) / np.diff(df[zvar], prepend=np.nan)) ** 2
C:\Users\nixo1\A

row
11281     2.630000
11282     2.700000
11283     2.780000
11284     2.950000
11285     3.110000
           ...    
11470    52.200001
11471    52.380001
11472    52.549999
11473    52.750000
11474    52.900002
Name: pressure, Length: 194, dtype: float32
row
11475    52.900002
11476    52.860001
11477    52.730000
11478    52.570000
11479    52.410000
           ...    
11647     2.500000
11648     2.310000
11649     2.170000
11650     2.040000
11651     2.000000
Name: pressure, Length: 177, dtype: float32
row
11652     2.000000
11653     2.060000
11654     2.190000
11655     2.410000
11656     2.770000
           ...    
11843    51.450001
11844    51.630001
11845    51.779999
11846    51.880001
11847    51.889999
Name: pressure, Length: 196, dtype: float32
row
11848    51.889999
11849    51.790001
11850    51.689999
11851    51.509998
11852    51.270000
           ...    
12024     2.180000
12025     1.920000
12026     1.660000
12027     1.470000
12028     1.390000
Name: pressure, 

C:\Users\nixo1\AppData\Local\Temp\ipykernel_22600\3273644669.py:41: RuntimeWarning: invalid value encountered in sqrt
  pN2 = np.sqrt(9.81 / np.nanmean(df[mld_var]) * np.diff(df[mld_var], prepend=np.nan) / np.diff(df[zvar], prepend=np.nan)) ** 2
C:\Users\nixo1\AppData\Local\Temp\ipykernel_22600\3273644669.py:41: RuntimeWarning: invalid value encountered in sqrt
  pN2 = np.sqrt(9.81 / np.nanmean(df[mld_var]) * np.diff(df[mld_var], prepend=np.nan) / np.diff(df[zvar], prepend=np.nan)) ** 2
C:\Users\nixo1\AppData\Local\Temp\ipykernel_22600\3273644669.py:41: RuntimeWarning: invalid value encountered in sqrt
  pN2 = np.sqrt(9.81 / np.nanmean(df[mld_var]) * np.diff(df[mld_var], prepend=np.nan) / np.diff(df[zvar], prepend=np.nan)) ** 2


row
15546    52.240002
15547    52.189999
15548    52.080002
15549    52.000000
15550    51.889999
           ...    
15737     2.900000
15738     2.590000
15739     2.240000
15740     1.980000
15741     1.870000
Name: pressure, Length: 196, dtype: float32
row
15742     1.870000
15743     1.880000
15744     1.970000
15745     1.980000
15746     2.060000
           ...    
15927    51.400002
15928    51.590000
15929    51.919998
15930    52.180000
15931    52.410000
Name: pressure, Length: 190, dtype: float32
row
15932    52.410000
15933    52.380001
15934    52.259998
15935    52.029999
15936    51.889999
           ...    
16124     3.320000
16125     3.170000
16126     3.080000
16127     3.050000
16128     2.960000
Name: pressure, Length: 197, dtype: float32
row
16129     2.960000
16130     3.040000
16131     3.080000
16132     3.280000
16133     3.510000
           ...    
16293    50.139999
16294    50.349998
16295    50.630001
16296    50.919998
16297    51.150002
Name: pressure, 

...

row
160173     3.080000
160174     3.150000
160175     3.260000
160176     3.450000
160177     3.650000
            ...    
160287    32.209999
160288    32.410000
160289    32.610001
160290    32.770000
160291    32.869999
Name: pressure, Length: 119, dtype: float32
row
160292    32.869999
160293    32.860001
160294    32.820000
160295    32.680000
160296    32.500000
            ...    
160422     4.010000
160423     3.760000
160424     3.610000
160425     3.470000
160426     3.460000
Name: pressure, Length: 135, dtype: float32
row
160427     3.460000
160428     3.470000
160429     3.570000
160430     3.720000
160431     3.920000
            ...    
160539    30.820000
160540    31.030001
160541    31.209999
160542    31.400000
160543    31.469999
Name: pressure, Length: 117, dtype: float32
row
160544    31.469999
160545    31.469999
160546    31.400000
160547    31.270000
160548    31.090000
            ...    
160665     3.700000
160666     3.490000
160667     3.350000
160668     3

C:\Users\nixo1\AppData\Local\Temp\ipykernel_22600\3273644669.py:41: RuntimeWarning: invalid value encountered in sqrt
  pN2 = np.sqrt(9.81 / np.nanmean(df[mld_var]) * np.diff(df[mld_var], prepend=np.nan) / np.diff(df[zvar], prepend=np.nan)) ** 2


row
162328    31.250000
162329    31.160000
162330    31.040001
162331    30.870001
162332    30.700001
            ...    
162457     3.810000
162458     3.620000
162459     3.470000
162460     3.380000
162461     3.330000
Name: pressure, Length: 134, dtype: float32
row
162462     3.330000
162463     3.390000
162464     3.480000
162465     3.660000
162466     3.870000
            ...    
162565    29.510000
162566    29.730000
162567    29.879999
162568    30.020000
162569    30.059999
Name: pressure, Length: 108, dtype: float32
row
162570    30.059999
162571    30.049999
162572    29.930000
162573    29.780001
162574    29.549999
            ...    
162696     3.780000
162697     3.630000
162698     3.500000
162699     3.460000
162700     3.450000
Name: pressure, Length: 131, dtype: float32
row
162701     3.450000
162702     3.550000
162703     3.690000
162704     3.920000
162705     4.150000
            ...    
162801    28.870001
162802    29.049999
162803    29.290001
162804    29

C:\Users\nixo1\AppData\Local\Temp\ipykernel_22600\3273644669.py:41: RuntimeWarning: invalid value encountered in sqrt
  pN2 = np.sqrt(9.81 / np.nanmean(df[mld_var]) * np.diff(df[mld_var], prepend=np.nan) / np.diff(df[zvar], prepend=np.nan)) ** 2


row
167137    28.559999
167138    28.500000
167139    28.420000
167140    28.270000
167141    28.110001
            ...    
167244     4.190000
167245     4.000000
167246     3.850000
167247     3.770000
167248     3.750000
Name: pressure, Length: 112, dtype: float32
row
167249     3.750000
167250     3.810000
167251     3.920000
167252     4.110000
167253     4.350000
            ...    
167348    29.150000
167349    29.350000
167350    29.549999
167351    29.660000
167352    29.680000
Name: pressure, Length: 104, dtype: float32
row
167353    29.680000
167354    29.590000
167355    29.469999
167356    29.280001
167357    29.090000
            ...    
167464     4.080000
167465     3.900000
167466     3.710000
167467     3.600000
167468     3.520000
Name: pressure, Length: 116, dtype: float32
row
167469     3.520000
167470     3.560000
167471     3.640000
167472     3.790000
167473     3.990000
            ...    
167570    28.959999
167571    29.200001
167572    29.420000
167573    29